In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
import zipfile
import os

# Path to your ZIP file

zip_file_path = '/content/midi_song.zip'
extracted_folder_path = '/content/midi_songs'

# Create the directory if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# List the contents of the top-level extracted folder
extracted_files = os.listdir(extracted_folder_path)
print("Top-level extracted files and folders:", extracted_files)



Top-level extracted files and folders: ['midi_songs']


Unzipping the mid_songs file and parsing the songs in midi format

# ***Preparing the data***

In [ ]:
import glob
from music21 import converter, instrument, note, chord

# Path to the subfolder containing the MIDI files
subfolder_path = os.path.join(extracted_folder_path, 'midi_songs')  # Replace with actual subfolder name

# List files in the subfolder to verify
extracted_files = os.listdir(subfolder_path)
print("Files in subfolder:", extracted_files)

notes = []

# Process each MIDI file in the subfolder
for file in glob.glob(os.path.join(subfolder_path, "*.mid")):
    print(f"Processing file: {file}")
    try:
        midi = converter.parse(file)
        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi)
        if parts:  # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        if not notes_to_parse:
            print(f"No notes to parse in file: {file}")

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    except Exception as e:
        print(f"Error processing file {file}: {e}")

print("Total notes:", len(notes))


Files in subfolder: ['FF4.mid', 'fortresscondor.mid', 'Finalfantasy6fanfarecomplete.mid', 'FFIXQuMarshP.mid', 'ff4pclov.mid', 'ff4-town.mid', 'pkelite4.mid', 'Fyw_piano.mid', 'sandy.mid', 'ff8-lfp.mid', 'Fiend_Battle_(Piano).mid', 'Finalfantasy5gilgameshp.mid', 'cosmo.mid', 'FF8_Shuffle_or_boogie_pc.mid', 'great_war.mid', 'ff4-airship.mid', 'z_aeristhemepiano.mid', 'Cids.mid', 'Gold_Silver_Rival_Battle.mid', 'ff4-fight1.mid', 'ff11_awakening_piano.mid', 'FFVII_BATTLE.mid', 'Rydia_pc.mid', 'costadsol.mid', 'Rachel_Piano_tempofix.mid', 'Eternal_Harvest.mid', '8.mid', 'decisive.mid', 'ff7themep.mid', 'Oppressed.mid', 'Fierce_Battle_(Piano).mid', 'VincentPiano.mid', 'thoughts.mid', 'dayafter.mid', 'balamb.mid', 'ff6shap.mid', 'Final_Fantasy_7_-_Judgement_Day_Piano.mid', 'braska.mid', '0fithos.mid', 'ff4_piano_collections-main_theme.mid', 'thenightmarebegins.mid', 'sobf.mid', 'rufus.mid', 'Suteki_Da_Ne_(Piano_Version).mid', 'figaro.mid', 'caitsith.mid', 'dontbeafraid.mid', 'redwings.mid', '

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Pok\xe9mon: Elite Four (Piano)'>; getting generic Instrument
  warnings.warn(


Processing file: /content/midi_songs/midi_songs/Fyw_piano.mid
Processing file: /content/midi_songs/midi_songs/sandy.mid
Processing file: /content/midi_songs/midi_songs/ff8-lfp.mid
Processing file: /content/midi_songs/midi_songs/Fiend_Battle_(Piano).mid
Processing file: /content/midi_songs/midi_songs/Finalfantasy5gilgameshp.mid
Processing file: /content/midi_songs/midi_songs/cosmo.mid
Processing file: /content/midi_songs/midi_songs/FF8_Shuffle_or_boogie_pc.mid
Processing file: /content/midi_songs/midi_songs/great_war.mid
Processing file: /content/midi_songs/midi_songs/ff4-airship.mid
Processing file: /content/midi_songs/midi_songs/z_aeristhemepiano.mid
Processing file: /content/midi_songs/midi_songs/Cids.mid
Processing file: /content/midi_songs/midi_songs/Gold_Silver_Rival_Battle.mid
Processing file: /content/midi_songs/midi_songs/ff4-fight1.mid
Processing file: /content/midi_songs/midi_songs/ff11_awakening_piano.mid
Processing file: /content/midi_songs/midi_songs/FFVII_BATTLE.mid
Proce

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Martin Rosok Copyright \xa9 1998'>; getting generic Instrument
  warnings.warn(


Processing file: /content/midi_songs/midi_songs/Oppressed.mid
Processing file: /content/midi_songs/midi_songs/Fierce_Battle_(Piano).mid
Processing file: /content/midi_songs/midi_songs/VincentPiano.mid
Processing file: /content/midi_songs/midi_songs/thoughts.mid
Processing file: /content/midi_songs/midi_songs/dayafter.mid
Processing file: /content/midi_songs/midi_songs/balamb.mid
Processing file: /content/midi_songs/midi_songs/ff6shap.mid
Processing file: /content/midi_songs/midi_songs/Final_Fantasy_7_-_Judgement_Day_Piano.mid
Processing file: /content/midi_songs/midi_songs/braska.mid
Processing file: /content/midi_songs/midi_songs/0fithos.mid
Processing file: /content/midi_songs/midi_songs/ff4_piano_collections-main_theme.mid
Processing file: /content/midi_songs/midi_songs/thenightmarebegins.mid
Processing file: /content/midi_songs/midi_songs/sobf.mid
Processing file: /content/midi_songs/midi_songs/rufus.mid
Processing file: /content/midi_songs/midi_songs/Suteki_Da_Ne_(Piano_Version).m

# ***Analyzing the data***

- analysing the note and chordes
- analysing the pitch of the music

In [ ]:
import numpy as np
from keras.utils import to_categorical

sequence_length = 100

# Get all pitch names
pitchnames = sorted(set(item for item in notes))
n_vocab = len(pitchnames)  # Number of unique notes

# Create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# Create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# Reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# Normalize input
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output, num_classes=n_vocab)

print("Total notes:", len(notes))
print("Number of patterns:", n_patterns)
print("Sample network_output:", network_output[:5])  # Print a sample of the network_output
print("Network output length:", len(network_output))
print("Input shape:", network_input.shape)
print("Output shape:", network_output.shape)


Total notes: 45976
Number of patterns: 45876
Sample network_output: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Network output length: 45876
Input shape: (45876, 100, 1)
Output shape: (45876, 326)


# ***Define the LSTM Model***

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

# Define the model
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# ***Trainig the Model***

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.summary()  # Print model summary to verify the architecture


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 100, 512)            │       1,052,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 100, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 100, 512)            │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 100, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 512)                 │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 326)                 │          83,782 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 326)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,466,182 (20.85 MB)

 Trainable params: 5,466,182 (20.85 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

# Prepare the data
sequence_length = 100
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
n_vocab = len(note_to_int)

# Create input sequences and the corresponding outputs
network_input = []
network_output = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# Reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)

# Define the model
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Set up ModelCheckpoint to save the best model
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

# Train the model
# Train the model with fewer epochs
model.fit(network_input, network_output, epochs=25, batch_size=64, callbacks=callbacks_list)



Epoch 1/25
717/717 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 4.7518
Epoch 1: loss improved from inf to 4.62689, saving model to weights-improvement-01-4.6269-bigger.keras
717/717 ━━━━━━━━━━━━━━━━━━━━ 2954s 4s/step - loss: 4.7516
Epoch 2/25
717/717 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 4.5311
Epoch 2: loss improved from 4.62689 to 4.52448, saving model to weights-improvement-02-4.5245-bigger.keras
717/717 ━━━━━━━━━━━━━━━━━━━━ 2762s 4s/step - loss: 4.5311
Epoch 3/25
717/717 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 4.4984
Epoch 3: loss improved from 4.52448 to 4.49070, saving model to weights-improvement-03-4.4907-bigger.keras
717/717 ━━━━━━━━━━━━━━━━━━━━ 3171s 4s/step - loss: 4.4984
Epoch 4/25
717/717 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 4.4682
Epoch 4: loss improved from 4.49070 to 4.46473, saving model to weights-improvement-04-4.4647-bigger.keras
717/717 ━━━━━━━━━━━━━━━━━━━━ 2931s 4s/step - loss: 4.4682
Epoch 5/25
717/717 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 4.4277
Epoch 5: loss 